# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [9]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [18]:
# 6328 people viewed, 1860 people clicked

In [10]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [21]:
#Your code here
df['clicks'] = df['action'].map(lambda x: 1 if x == 'click' else 0)

In [45]:
# check
df[['action', 'clicks']][:10]

,action,clicks
0,view,0
1,view,0
2,view,0
3,view,0
4,view,0
5,view,0
6,view,0
7,view,0
8,click,1
9,view,0


In [23]:
control = df[df['group'] == 'control']['clicks']
control

3       0
9       0
23      0
24      0
25      0
       ..
8178    0
8181    0
8182    1
8186    0
8187    0
Name: clicks, Length: 4264, dtype: int64

In [48]:
controlclicks = sum(control)

In [47]:
# as individuals counter twice if view and click
controlsize = len(control)-sum(control)

In [56]:
clickratecontrol = controlclicks/controlsize *100
clickratecontrol

27.971188475390157

In [27]:
experiment = df[df['group'] == 'experiment']['clicks']
experiment

0       0
1       0
2       0
4       0
5       0
       ..
8179    0
8180    1
8183    0
8184    0
8185    0
Name: clicks, Length: 3924, dtype: int64

In [57]:
experimentclicks = sum(experiment)

# as individuals counter twice if view and click
experimentsize = len(experiment)-sum(experiment)

In [58]:
clickrateexperiment = experimentclicks/experimentsize *100
clickrateexperiment

30.97463284379172

In [32]:
# H0 no statistical difference in click rate
# H1 new page is effective, click rate is > for experimental group

In [70]:
#Use pivot table to set index as id and avoid double counting views and clicks
df['count'] = 1

control = df[df['group']=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df['group'] =='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [67]:
control

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [73]:
print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control['click'].sum(), experiment['click'].sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control['click'].mean(), experiment['click'].mean()))

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


In [77]:
import scipy.stats as stats

t_val, p_val = stats.ttest_ind(control['click'], experiment['click'], equal_var = False)

p_val/2

0.004466402814337101

In [78]:
# With alpha = 0.05, p_val < alpha so reject H0 and accept H1

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [82]:
#Your code here
expectedclicks = len(experiment)*clickratecontrol/100
expectedclicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [85]:
#Your code here
actualclicks = sum(experiment['click'])
actualclicks

928.0

In [92]:
# var = n * p * (1-p)
n = len(experiment)
p = clickratecontrol/100

var = n * p * (1-p)
sd = np.sqrt(var)
sd

24.568547907005815

In [99]:
diff = actualclicks - expectedclicks

In [102]:
z = diff/sd
z

3.6625360854823588

In [98]:
#3.66 sd away

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [105]:
#Your code here
import scipy.stats as stats

# survival function sf is 1 - cdf
p_val = stats.norm.sf(z)
p_val

0.00012486528006951198

In [106]:

pval = 1 - stats.norm.cdf(z)
pval

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [107]:
# Much smaller p-value, but in any case H0 rejected

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.